# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments               Customer  \
0  Oct 26 2022 11:06AM  249412        15  30011714  Alliance Pharma, Inc.   
1  Oct 26 2022 11:06AM  249412        15  30011715  Alliance Pharma, Inc.   
2  Oct 26 2022 11:06AM  249412        15  30011716  Alliance Pharma, Inc.   
3  Oct 26 2022 11:06AM  249412        15  30011717  Alliance Pharma, Inc.   
4  Oct 26 2022 11:06AM  249412        15  30011718  Alliance Pharma, Inc.   
5  Oct 26 2022 11:06AM  249412        15  30011719  Alliance Pharma, Inc.   
6  Oct 26 2022 11:06AM  249412        15  30011721  Alliance Pharma, Inc.   
7  Oct 26 2022 11:06AM  249412        15  30011722  Alliance Pharma, Inc.   
8  Oct 26 2022 11:06AM  249412        15  30011723  Alliance Pharma, Inc.   
9  Oct 26 2022 11:06AM  249412        15  30011724  Alliance Pharma, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
24  249418       Released          1
25  249419       Released          1
26  249420      Executing          7
27  249420       Released          3
28  249422       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249414                NaN        2.0       1.0
249418                NaN        NaN       1.0
249419                NaN        NaN       1.0
249420                NaN        7.0       3.0
249422                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249302                0.0        3.0       4.0
249344                0.0        0.0       1.0
249372                0.0        0.0       1.0
249382                0.0        0.0       1.0
249386                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249302                  0          3         4
249344                  0          0         1
249372                  0          0         1
249382                  0          0         1
249386                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249302          0          3         4
1               249344          0          0         1
2               249372          0          0         1
3               249382          0          0         1
4               249386          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               249302                   3         4
1               249344                             1
2               249372                             1
3               249382                             1
4               249386                             1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Oct 26 2022 11:06AM  249412        15            Alliance Pharma, Inc.
23  Oct 26 2022 10:55AM  249422        19           GCH Granules USA, Inc.
24  Oct 26 2022 10:52AM  249420        15      Virtus Pharmaceuticals, LLC
34  Oct 26 2022 10:40AM  249419        16   Sartorius Bioprocess Solutions
35  Oct 26 2022 10:38AM  249418        12                 SD Head USA, LLC
36  Oct 26 2022 10:36AM  249414        15           Mizner Bioscience, LLC
39  Oct 26 2022 10:35AM  249413        15  Brookfield Pharmaceuticals, LLC
58  Oct 26 2022 10:30AM  249410        10                ISDIN Corporation
64  Oct 26 2022 10:29AM  249411        10                ISDIN Corporation
73  Oct 26 2022 10:26AM  249403        10                ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Oct 26 2022 11:06AM  249412        15            Alliance Pharma, Inc.   
1  Oct 26 2022 10:55AM  249422        19           GCH Granules USA, Inc.   
2  Oct 26 2022 10:52AM  249420        15      Virtus Pharmaceuticals, LLC   
3  Oct 26 2022 10:40AM  249419        16   Sartorius Bioprocess Solutions   
4  Oct 26 2022 10:38AM  249418        12                 SD Head USA, LLC   
5  Oct 26 2022 10:36AM  249414        15           Mizner Bioscience, LLC   
6  Oct 26 2022 10:35AM  249413        15  Brookfield Pharmaceuticals, LLC   
7  Oct 26 2022 10:30AM  249410        10                ISDIN Corporation   
8  Oct 26 2022 10:29AM  249411        10                ISDIN Corporation   
9  Oct 26 2022 10:26AM  249403        10                ISDIN Corporation   

  Completed Executing  Released  
0                            23  
1                             1  
2                   7         3  
3                             1  
4                             1  
5                   2         1  
6                  10         9  
7                             6  
8                             9  
9                            48

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Oct 26 2022 11:06AM  249412        15            Alliance Pharma, Inc.   
1  Oct 26 2022 10:55AM  249422        19           GCH Granules USA, Inc.   
2  Oct 26 2022 10:52AM  249420        15      Virtus Pharmaceuticals, LLC   
3  Oct 26 2022 10:40AM  249419        16   Sartorius Bioprocess Solutions   
4  Oct 26 2022 10:38AM  249418        12                 SD Head USA, LLC   
5  Oct 26 2022 10:36AM  249414        15           Mizner Bioscience, LLC   
6  Oct 26 2022 10:35AM  249413        15  Brookfield Pharmaceuticals, LLC   
7  Oct 26 2022 10:30AM  249410        10                ISDIN Corporation   
8  Oct 26 2022 10:29AM  249411        10                ISDIN Corporation   
9  Oct 26 2022 10:26AM  249403        10                ISDIN Corporation   

   Released Executing Completed  
0        23                      
1         1                      
2         3         7            
3         1                      
4         1                      
5         1         2            
6         9        10            
7         6                      
8         9                      
9        48

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 26 2022 11:06AM  249412        15           Alliance Pharma, Inc.   
1  Oct 26 2022 10:55AM  249422        19          GCH Granules USA, Inc.   
2  Oct 26 2022 10:52AM  249420        15     Virtus Pharmaceuticals, LLC   
3  Oct 26 2022 10:40AM  249419        16  Sartorius Bioprocess Solutions   
4  Oct 26 2022 10:38AM  249418        12                SD Head USA, LLC   

   Released Executing Completed  
0        23                      
1         1                      
2         3         7            
3         1                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 26 2022 11:06AM  249412        15           Alliance Pharma, Inc.   
1  Oct 26 2022 10:55AM  249422        19          GCH Granules USA, Inc.   
2  Oct 26 2022 10:52AM  249420        15     Virtus Pharmaceuticals, LLC   
3  Oct 26 2022 10:40AM  249419        16  Sartorius Bioprocess Solutions   
4  Oct 26 2022 10:38AM  249418        12                SD Head USA, LLC   

   Released  Executing  Completed  
0        23        NaN        NaN  
1         1        NaN        NaN  
2         3        7.0        NaN  
3         1        NaN        NaN  
4         1        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 26 2022 11:06AM  249412        15           Alliance Pharma, Inc.   
1  Oct 26 2022 10:55AM  249422        19          GCH Granules USA, Inc.   
2  Oct 26 2022 10:52AM  249420        15     Virtus Pharmaceuticals, LLC   
3  Oct 26 2022 10:40AM  249419        16  Sartorius Bioprocess Solutions   
4  Oct 26 2022 10:38AM  249418        12                SD Head USA, LLC   

   Released  Executing  Completed  
0        23        0.0        0.0  
1         1        0.0        0.0  
2         3        7.0        0.0  
3         1        0.0        0.0  
4         1        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1745770        97       11.0        2.0
12          249418         1        0.0        0.0
15         1246961        40       22.0        0.0
16          249419         1        0.0        0.0
19          748176         3        0.0        0.0
21         1246960         5        0.0        0.0
22          249408         1        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1745770        97       11.0        2.0
1        12   249418         1        0.0        0.0
2        15  1246961        40       22.0        0.0
3        16   249419         1        0.0        0.0
4        19   748176         3        0.0        0.0
5        21  1246960         5        0.0        0.0
6        22   249408         1        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        97       11.0        2.0
1        12         1        0.0        0.0
2        15        40       22.0        0.0
3        16         1        0.0        0.0
4        19         3        0.0        0.0
5        21         5        0.0        0.0
6        22         1        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   97.0
1        12  Released    1.0
2        15  Released   40.0
3        16  Released    1.0
4        19  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16   19   21   22
Status                                        
Completed   2.0  0.0   0.0  0.0  0.0  0.0  0.0
Executing  11.0  0.0  22.0  0.0  0.0  0.0  0.0
Released   97.0  1.0  40.0  1.0  3.0  5.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16   19   21   22
0          Completed   2.0  0.0   0.0  0.0  0.0  0.0  0.0
1          Executing  11.0  0.0  22.0  0.0  0.0  0.0  0.0
2           Released  97.0  1.0  40.0  1.0  3.0  5.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16   19   21   22
0  Completed   2.0  0.0   0.0  0.0  0.0  0.0  0.0
1  Executing  11.0  0.0  22.0  0.0  0.0  0.0  0.0
2   Released  97.0  1.0  40.0  1.0  3.0  5.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()